In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers

IMAGE_SIZE=256
BATCH_SIZE=32


In [2]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


Found 3833 files belonging to 2 classes.


In [3]:
class_name=dataset.class_names
class_name

['with_mask', 'without_mask']

In [4]:
# for image_batch, label_batch in dataset.take(1):

# test and validation split
# 80% training , 10% test 10%  


In [18]:
def get_dataset_partitions(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle = True , shuffle_size=1000):
    ds_size = len(ds)
    if shuffle:
        ds.shuffle(shuffle_size,seed=12)
    #sizes
    train_size = int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    #datasets
    train_ds=dataset.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    
    return train_ds , val_ds , test_ds


In [19]:
train_ds,val_ds,test_ds = get_dataset_partitions(dataset)

In [21]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Resize and rescale

In [22]:
resize_and_rescale=tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
        layers.experimental.preprocessing.Rescaling(0.1/255)
    ]
)

Data Augmentation

In [23]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [24]:
input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,3)
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(3,activation='softmax')
])
model.build(input_shape=input_shape)

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (32, 256, 256, 3)         0         
_________________________________________________________________
sequential_1 (Sequential)    (32, 256, 256, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (32, 254, 254, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 127, 127, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 125, 125, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 62, 62, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 60, 60, 64)         

In [26]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
    
)

In [27]:
model.fit(train_ds,epochs=20,batch_size=BATCH_SIZE,verbose=1,validation_data=val_ds)

Epoch 1/20
96/96 [==============================] - 608s 3s/step - loss: 0.7305 - accuracy: 0.5055 - val_loss: 0.6905 - val_accuracy: 0.4935
Epoch 2/20
96/96 [==============================] - 198s 2s/step - loss: 0.6175 - accuracy: 0.6296 - val_loss: 0.4088 - val_accuracy: 0.8258
Epoch 3/20
96/96 [==============================] - 274s 3s/step - loss: 0.3043 - accuracy: 0.8854 - val_loss: 0.3537 - val_accuracy: 0.8675
Epoch 4/20
96/96 [==============================] - 163s 2s/step - loss: 0.2917 - accuracy: 0.8841 - val_loss: 0.3145 - val_accuracy: 0.8675
Epoch 5/20
96/96 [==============================] - 295s 3s/step - loss: 0.2706 - accuracy: 0.9033 - val_loss: 0.2436 - val_accuracy: 0.9066
Epoch 6/20
96/96 [==============================] - 181s 2s/step - loss: 0.2651 - accuracy: 0.9033 - val_loss: 0.2859 - val_accuracy: 0.8929
Epoch 7/20
96/96 [==============================] - 161s 2s/step - loss: 0.2813 - accuracy: 0.8926 - val_loss: 0.2569 - val_accuracy: 0.9066
Epoch 8/20
96

In [28]:
import os.path

In [29]:
if os.path.isfile('models/maskModel.h5') is False:
    model.save('models/maskModel.h5')

In [31]:
from tensorflow.keras.models import load_model
newModel=load_model('models/maskModel.h5')

In [32]:
newModel

In [33]:
model